In [12]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import uuid
import time

In [13]:
# Set the seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
# Define the path to your raw data
raw_data_dirs = [r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\1. Eczema 1677", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\2. Melanoma 15.75k", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\3. Atopic Dermatitis - 1.25k", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\4. Basal Cell Carcinoma (BCC) 3323", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\5. Melanocytic Nevi (NV) - 7970", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\6. Benign Keratosis-like Lesions (BKL) 2624", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\7. Psoriasis pictures Lichen Planus and related diseases - 2k", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\8. Seborrheic Keratoses and other Benign Tumors - 1.8k", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k", r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\10. Warts Molluscum and other Viral Infections - 2103"]
output_data_dir = r"C:\Users\HP\Downloads\WaysAhead\archive\IMG_CLASSES\output"

# Create output_data_dir if it doesn't exist
os.makedirs(output_data_dir, exist_ok=True)

In [15]:
# Define parameters
img_width, img_height = 256, 256
batch_size = 32
epochs = 10
test_size = 0.20  # 20% of the data will be used for testing

# List all files in the raw data directories
raw_image_files = []

for data_dir in raw_data_dirs:
    raw_image_files.extend([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])

# Extract labels from the directory names
labels = [os.path.basename(os.path.dirname(f)) for f in raw_image_files]

In [16]:
print(len(raw_image_files))
print(labels)


27153
['1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1. Eczema 1677', '1.

In [17]:
# Convert labels to integer indices
label_to_index = {label: idx for idx, label in enumerate(set(labels))}
indexed_labels = [label_to_index[label] for label in labels]

In [18]:
print(label_to_index)
print(indexed_labels)

{'6. Benign Keratosis-like Lesions (BKL) 2624': 0, '3. Atopic Dermatitis - 1.25k': 1, '1. Eczema 1677': 2, '2. Melanoma 15.75k': 3, '4. Basal Cell Carcinoma (BCC) 3323': 4, '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k': 5, '10. Warts Molluscum and other Viral Infections - 2103': 6, '8. Seborrheic Keratoses and other Benign Tumors - 1.8k': 7, '5. Melanocytic Nevi (NV) - 7970': 8, '7. Psoriasis pictures Lichen Planus and related diseases - 2k': 9}
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [19]:
# Split the data into training and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(raw_image_files, indexed_labels, test_size=test_size, random_state=42)

# Create separate folders for training and testing data
train_data_dir = os.path.join(output_data_dir, "train")
test_data_dir = os.path.join(output_data_dir, "test")

# Ensure output directories exist, create them if necessary
for directory in [train_data_dir, test_data_dir]:
    os.makedirs(directory, exist_ok=True)

In [20]:
print(train_labels)
print(test_labels)
print(len(train_files))
print(len(test_files))

[3, 4, 0, 0, 8, 5, 8, 8, 4, 8, 5, 6, 2, 4, 0, 3, 8, 4, 9, 2, 2, 0, 1, 2, 2, 3, 8, 1, 6, 8, 4, 3, 8, 8, 4, 8, 3, 8, 8, 2, 9, 4, 8, 3, 8, 4, 4, 8, 8, 0, 2, 4, 0, 8, 1, 4, 9, 4, 9, 5, 4, 4, 8, 9, 3, 3, 3, 1, 4, 7, 3, 8, 3, 0, 2, 6, 4, 5, 9, 6, 1, 9, 1, 7, 8, 3, 8, 3, 8, 4, 8, 4, 4, 8, 4, 9, 9, 8, 5, 7, 5, 9, 1, 5, 2, 2, 0, 4, 3, 0, 1, 8, 8, 8, 8, 1, 7, 8, 3, 0, 6, 0, 8, 6, 5, 4, 3, 8, 9, 4, 2, 2, 9, 8, 8, 8, 6, 4, 8, 0, 8, 5, 1, 0, 3, 3, 6, 4, 4, 2, 7, 0, 4, 5, 3, 6, 2, 8, 4, 9, 9, 3, 4, 9, 6, 3, 6, 8, 0, 8, 4, 3, 7, 8, 8, 3, 2, 3, 7, 8, 8, 8, 7, 8, 5, 8, 0, 7, 3, 8, 1, 9, 5, 2, 4, 2, 3, 8, 8, 9, 9, 3, 8, 6, 8, 3, 5, 8, 0, 6, 8, 3, 4, 0, 9, 6, 5, 6, 9, 1, 3, 4, 2, 8, 3, 8, 8, 8, 8, 8, 1, 8, 2, 4, 2, 8, 2, 0, 3, 8, 9, 8, 6, 3, 2, 3, 2, 3, 8, 7, 8, 9, 2, 4, 3, 8, 6, 3, 0, 3, 9, 3, 8, 8, 9, 3, 3, 8, 9, 3, 8, 0, 2, 6, 0, 3, 1, 4, 4, 0, 1, 9, 3, 8, 8, 2, 8, 9, 2, 8, 8, 2, 3, 3, 8, 7, 6, 3, 4, 6, 9, 6, 3, 9, 0, 9, 4, 8, 6, 8, 9, 8, 8, 8, 0, 9, 0, 3, 0, 1, 5, 7, 5, 6, 3, 9, 2, 8, 6, 8, 7, 3, 6, 

In [21]:
# Data cleaning and split
for file, label in tqdm(zip(train_files, train_labels), desc='Processing Training Data'):
    src_path = file
    dest_folder = os.path.join(train_data_dir, str(label))
    dest_path = os.path.join(dest_folder, os.path.basename(file))

    os.makedirs(dest_folder, exist_ok=True)

    try:
        # Read the image
        image = cv2.imread(file)
        
        #cv2.imshow("label", image)
        #cv2.waitKey(0)

        # Resize the image to the target size
        image = cv2.resize(image, (img_width, img_height))


        # Save the cleaned image
        if not cv2.imwrite(dest_path, image):
            raise Exception("Could not write image")
        
    except Exception as e:
        # Handle any errors that may occur during processing
        print(f"Error processing {src_path}: {e}")


for file, label in tqdm(zip(test_files, test_labels), desc='Processing Testing Data'):
    src_path = file
    dest_folder = os.path.join(test_data_dir, str(label))
    dest_path = os.path.join(dest_folder, os.path.basename(file))

    os.makedirs(dest_folder, exist_ok=True)

    try:
        # Read the image
        image = cv2.imread(file)
        
        #cv2.imshow("label", image)
        #cv2.waitKey(0)
        
        # Resize the image to the target size
        image = cv2.resize(image, (img_width, img_height))

        

        # Save the cleaned image
        if not cv2.imwrite(dest_path, image):
            raise Exception("Could not write image")
    except Exception as e:
        # Handle any errors that may occur during processing
        print(f"Error processing {src_path}: {e}")


Processing Training Data: 21722it [03:03, 118.66it/s]
Processing Testing Data: 5431it [00:48, 111.47it/s]


In [22]:
# Use pre-trained model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

# Create your own model
model = Sequential()

model.add(base_model)


model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(len(set(labels)), activation='softmax'))  # Output layer with softmax activation for multiclass


# Compile the model
def schedule(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(schedule)
opt = SGD(learning_rate=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Rescaling for the test set
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate the training set
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'  # Use 'sparse' for multiclass classification
)

# Generate the test set
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'  # Use 'sparse' for multiclass classification
)

Found 21722 images belonging to 10 classes.
Found 5431 images belonging to 10 classes.


In [24]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/10


678/678 [==============================] - 2089s 3s/step - loss: 1.1711 - accuracy: 0.5677 - val_loss: 1.3119 - val_accuracy: 0.5425
Epoch 2/10
678/678 [==============================] - 2066s 3s/step - loss: 0.9611 - accuracy: 0.6424 - val_loss: 1.0902 - val_accuracy: 0.6148
Epoch 3/10
678/678 [==============================] - 2080s 3s/step - loss: 0.8883 - accuracy: 0.6690 - val_loss: 1.1021 - val_accuracy: 0.6091
Epoch 4/10
678/678 [==============================] - 2073s 3s/step - loss: 0.8389 - accuracy: 0.6941 - val_loss: 0.9734 - val_accuracy: 0.6527
Epoch 5/10
678/678 [==============================] - 2070s 3s/step - loss: 0.7922 - accuracy: 0.7131 - val_loss: 0.9385 - val_accuracy: 0.6679
Epoch 6/10
678/678 [==============================] - 2055s 3s/step - loss: 0.7597 - accuracy: 0.7223 - val_loss: 0.9846 - val_accuracy: 0.6537
Epoch 7/10
678/678 [==============================] - 2059s 3s/step - loss: 0.7233 - accuracy: 0.7355 - val_loss: 1.0304 - val_accurac

In [25]:
# Save the model
model.save('multiclass_rash_detection_model.h5')

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
t1 = cv2.imread("sample3.jpg")
t1 = cv2.resize(t1, (img_width, img_height))
t1_a = image.img_to_array(t1)
t1_a = np.expand_dims(t1_a, axis=0)
t1_a/=255.0

predictions=model.predict(t1_a)
print("predictions: ")
print(predictions)

In [ ]:
max_val = np.max(predictions)
print(max_val)

c=0
for i in predictions[0]:
    if i==max_val:
        break
    c=c+1

print(c)

In [ ]:
z=list(label_to_index.keys())
print(z)
print(labels[c])